In [1]:
from datetime import date
import pymongo as pm

In [ ]:
!brew services start mongodb-community@4.2

In [ ]:
!brew services stop mongodb-community@4.2

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
DB_NAME = 'jobscomp'
db = client[DB_NAME]
col_name = 'descriptions'
collection = db[col_name]
db.list_collection_names()

[]

In [ ]:
sources = list("\
            http://www.careerbuilder.com \
            http://www.indeed.com \
            https://www.linkedin.com \
            http://careers.stackoverflow.com \
            http://www.glassdoor.com \
            https://angel.co \
            http://www.startuphire.com \
            http://www.monster.com \
            ".split())
sources

In [ ]:
url = 'https://www.careerbuilder.com/jobs?utf8=✓&keywords=' + job_title + '&location=' + zip_code

'https://www.careerbuilder.com/jobs?keywords=%E2%80%9Cdata+scientist%E2%80%9D&location=20005&page_number=2'
url

In [3]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "jobs"

    def start_requests(self):
        pages = 1
        urls = [
            'https://www.careerbuilder.com/jobs?keywords=%E2%80%9Cdata+scientist%E2%80%9D&location=20005&page_number=' 
            + str(page) for page in range(1, pages + 1)
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        
        # extract data in spider
        # another way to select all links is
        # response.xpath('//div[@id="jobs_collection"]//a[2]/@href').getall()
        # but need to add the first part of the url
        jobs_links = response.xpath('//div[@id="jobs_collection"]//a[2]/@href').getall()
        yield from response.follow_all(jobs_links, self.parse_job)
            
    def parse_job(self, response):
        yield {
            'title': response.xpath('//h1/text()').get(),
            'company': response.xpath('//*[@id="jdp-data"]/div[1]/div[2]/div/div[1]/div[1]/span/text()').get(),
            'description': ' '.join(response.xpath('//*[@id="jdp_description"]/div[1]/div[1]//text()').extract()),
            'date_scraped': date.today()
        }
            

In [9]:
!cd /Users/yegor/Desktop/data_scientist_comparer/scrapy/

In [10]:
!pwd

/Users/yegor/Desktop/data_scientist_comparer


In [11]:
# run spider in top directory of scrapy project
# and save results to `jobs.json` file
!scrapy crawl jobs -o jobs.jl

Scrapy 2.1.0 - no active project

Unknown command: crawl

Use "scrapy" to see available commands


In [ ]:
[a.attrib['href'] for a in response.xpath('//div[@id="jobs_collection"]//a')] 